## Laboratorio Final - Equipo Rocket

___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />

# Microestructuras y Sistemas de Trading
### Equipo ROCKET

## <font color="navy">  "PROYECTO FINAL"
    
* <font color="black">**INTEGRANTE 1 :**</font> Alejandro Preciado G
* <font color="black">**INTEGRANTE 2 :**</font> Luis Ontaneda
* <font color="black">**INTEGRANTE 3 :**</font> Esteban Navarrete
* <font color="black">**INTEGRANTE 4 :**</font> Pablo Vargas

### Propósito:

Diseñar una estrategia de trading con base a la reacción del precio ante el comunicado de indicadores macroeconómicos (Análisis fundamental).

#### Requisitos generales:

· Capital inicial: $100,000

· Riesgo por operación: $1,000

In [619]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
from collections import Counter
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import Boxplot as bx

In [620]:
#Cargamos los datos, y eliminamos los que no tienen valor
df = pd.read_csv("calendario_economico.csv") 
df.dropna(inplace = True)

In [621]:
data2=df.loc[df['Name'] == 'Retail Sales (MoM)']
RetailSales=data2.loc[df['Country'] == 'European Monetary Union']
RetailSales=RetailSales.reset_index(drop=True)

In [622]:
RetailSales['timestamp']=pd.to_datetime(RetailSales['timestamp'])
RetailSales['timestamp']=RetailSales['timestamp'].dt.tz_localize('UTC')

In [623]:
prueba = RetailSales[(RetailSales['timestamp'] > '2018-1-1 00:00:00') & 
                     (RetailSales['timestamp'] <= '2018-8-1 00:00:00')]

entrenamiento = RetailSales[(RetailSales['timestamp'] > '2009-5-1 00:00:00') & 
                     (RetailSales['timestamp'] <= '2017-12-1 00:00:00')]
prueba = prueba.reset_index(drop=True)
entrenamiento = entrenamiento.reset_index(drop=True)

In [624]:
#definimos escenarios por restricciones
def escenario(x):
    x['escenario']= x['actual']
    for i in range(len(x)):
        if x['actual'][i] >= x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i] = "A"
        elif x['actual'][i] >= x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="B"
        elif x['actual'][i] < x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i]="C"
        elif x['actual'][i] < x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="D"
    return x
    

In [625]:
def periodos(df,RS):
    # Cargamos los datos de los precios del Euro-Dolar
    # Checando valores en precios EUR/USD con misma hora que nuestro calendario económico
    # Data frame con los valores que coinciden las horas
    comparacionRS=df[df.timestamp.isin(RS.timestamp)]
    # Utilizamos compresión de listas para sacar un DataFrame de los indicadores para hacer match entre los comunicados del
    # indicador y los precios
    #Se busca desde la fecha y hora del indicador agarrar los precios de 30 minutos antes y 30 minutos despues (mas menos 6 periodos)
    periodosRS = [df.iloc[i-6:i+7]for i in comparacionRS.index]
    return periodosRS

In [626]:
def takeprofit(escenario):
    para=(RSescenarios[escenario]['pips alcistas'].max()*0.85-RSescenarios[escenario]['pips alcistas'].min()*1.15)/2
    takeprofitesc=RSescenarios[escenario]['pips alcistas'].min()*1.15+para
    return takeprofitesc
def StopLoss(escenario):
    para=(RSescenarios[escenario]['pips bajistas'].max()*0.85-RSescenarios[escenario]['pips bajistas'].min()*1.15)/2
    takeprofitesc=RSescenarios[escenario]['pips bajistas'].min()*1.15+para
    return takeprofitesc

In [627]:
# Con esta función se saca dirección, pips alcistas, pips bajistas y volatilidad de los precios de cada comunicado del indicador
# Se le mete el DataFrame de los comunicados de nuestro indicador y el match de los precios con el indicador
def finalDF(DataFrame,periodos):

    DataFrame['direccion']= DataFrame['actual']
    DataFrame['pips alcistas']=DataFrame['actual']
    DataFrame['pips bajistas']= DataFrame['actual']
    DataFrame['volatilidad']=DataFrame['actual']

    for i in range(len(DataFrame)):
        if periodos[i]["close"].iloc[-1] > periodos[i]["open"].iloc[6]:
            DataFrame['direccion'][i] = 1
        else:
            DataFrame['direccion'][i] = -1
                
        DataFrame['pips alcistas'][i] = (periodos[i]["high"].iloc[6:].max() - periodos[i]["open"].iloc[6])*10000
        DataFrame['pips bajistas'][i] = (periodos[i]["open"].iloc[6] - periodos[i]["low"].iloc[6:].min())*10000
        DataFrame['volatilidad'][i] = periodos[i]["high"].max()-periodos[i]["low"].min()
    return DataFrame




In [628]:
# Sacando cuales informes pertenecen dependiendo de escenario A, B, y C para checar comportamiento
def getescenario(DF):
    escA=DF[DF.escenario.isin(["A"])]
    escB=DF[DF.escenario.isin(["B"])]
    escC=DF[DF.escenario.isin(["C"])]
    escD=DF[DF.escenario.isin(["D"])]
    return escA,escB,escC,escD



In [629]:
def backtest(pruebabacktest,pruebaper,escenarios,df_decisiones):
    df_backtest=pd.DataFrame(np.zeros([len(pruebabacktest),8]),columns=["timestamp","escenario","operacion","volumen","resultado","pips",
                                                       "capital","balance final"])
    for i in range(len(pruebaper)):
        for e in range(len(escenarios)):
            if FinalDFRS["escenario"][i]==escenarios[e]:
                for j in range(int(len(pruebaper[i])/2),len(pruebaper[i])):
                    df_backtest["timestamp"][i] = pruebaper[i].iloc[j,0]
                    df_backtest["escenario"][i] = escenarios[e]
                    df_backtest["operacion"][i] = df_decisiones["operacion"][e]
                    df_backtest["volumen"][i] = df_decisiones.unidades[e]
                    if pruebaper[i].iloc[j,1]-df_decisiones.sl[e]/10000 >= pruebaper[i].iloc[j,3]:
                        df_backtest["resultado"][i] = "perdida"
                        df_backtest["pips"][i] = -df_decisiones.sl[e]
                        df_backtest["capital"][i] = (-df_decisiones.sl[e]/10000)*df_decisiones.unidades[e]

                        break
                    elif pruebaper[i].iloc[j,1]+df_decisiones.tp[e]/10000 <= pruebaper[i].iloc[j,2]:
                        df_backtest["resultado"][i] = "ganada"
                        df_backtest["pips"][i] = df_decisiones.tp[e]
                        df_backtest["capital"][i] = (df_decisiones.tp[e]/10000)*df_decisiones.unidades[e]

                        break
                    else:
                        df_backtest["resultado"][i] = "No se cumplio"
                        break

    df_backtest["balance final"][0] = 100000+df_backtest["capital"][0]   
    for i in range(1,len(df_backtest)):
        df_backtest["balance final"][i] = df_backtest["balance final"][i-1]+df_backtest["capital"][i]            

return df_backtest


SyntaxError: 'return' outside function (<ipython-input-629-7ac7d32a4bae>, line 32)

In [630]:
# utilizamos la función escenario para obtener cada comunicado del indicador económico si es A, B, C ó D dependiendo del actual,
# previous y consensus.
RS=escenario(entrenamiento)

In [631]:
df = pd.read_csv("precios_historicos_eurusd.csv") 
df['timestamp']=pd.to_datetime(df['timestamp'])
df['timestamp']=df['timestamp'].dt.tz_localize('UTC')
pruebaper = df[(df['timestamp'] > '2018-1-1 00:00:00') & 
                     (df['timestamp'] <= '2018-8-1 00:00:00')]
pruebaper = pruebaper.reset_index(drop=True)
df = df[(df['timestamp'] > '2009-5-1 00:00:00') & 
                     (df['timestamp'] <= '2017-12-1 00:00:00')]
df = df.reset_index(drop=True)

In [632]:
trainper = periodos(df,entrenamiento)
pruebaper = periodos(pruebaper,prueba)

In [633]:
# utilizamos la funcion finalDF() para obtener nuestro DataFrame final en retail sales.
FinalDFRS = finalDF(RS,trainper)
pruebabacktest = finalDF(escenario(prueba),prper)

In [634]:
takeProfitC = [takeprofit(i) for i in range(4)]
StopLossC = [StopLoss(i) for i in range(4)]
takeProfitV = StopLossC
StopLossV = takeProfitC

In [635]:
escenarios=["A","B","C","D"]
promalcistas=[FinalDFRS.loc[FinalDFRS["escenario"] == i]["pips alcistas"].mean() for i in escenarios]
prombajistas=[FinalDFRS.loc[FinalDFRS["escenario"] == i]["pips bajistas"].mean() for i in escenarios]

In [636]:
df_decisiones=pd.DataFrame({"escenario":escenarios})
df_decisiones["operacion"]=["compra" if promalcistas[i]>prombajistas[i] else "venta" for i in range(len(escenarios))]
df_decisiones["sl"]=[StopLossC[i] if df_decisiones["operacion"][i]=="compra" else takeProfitC[i] 
                  for i in range(len(escenarios))]
df_decisiones["tp"]=[takeProfitC[i] if df_decisiones["operacion"][i]=="compra" else StopLossC[i] 
                  for i in range(len(escenarios))]
df_decisiones["unidades"]=[855615,852515,943619,749344]

In [637]:
#Usamos la funcion Counter para contar cuantas veces se repiten las palabras
newcounter = Counter(FinalDFRS['escenario'])
escenariosRS = pd.DataFrame.from_dict(newcounter, orient='index').reset_index()
escenariosRS.columns = ['escenario', 'cantidad']
escenariosRS
###----------------cuantos escenarios hubo en cada uno A B C y D (RS)-------------------------

,escenario,cantidad
0,C,27
1,A,27
2,D,20
3,B,29


In [638]:
#para sacar el escenario "A"=RSescenarios[0]
#para sacar el escenario B =RSescenarios[1]
#para sacar el escenario C =RSescenarios[2]
#para sacar el escenario D =RSescenarios[3]
RSescenarios=getescenario(FinalDFRS)

In [639]:
df_backtest = backtest(pruebabacktest,pruebaper,escenarios,df_decisiones)
df_backtestentrenamiento = backtest(FinalDFRS,trainper,escenarios,df_decisiones)

In [640]:
df_backtest

In [641]:
df_backtestentrenamiento

In [642]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=df_backtest["timestamp"], y=df_backtest["capital"]))
fig.update_layout(title='Evolucion de Capital',
                   xaxis_title='Fecha',
                   yaxis_title='Capital')

TypeError: 'NoneType' object is not subscriptable

In [643]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=df_backtest["timestamp"], y=df_backtest["balance final"]))
fig.update_layout(title='Balance General',
                   xaxis_title='Fecha',
                   yaxis_title='Capital')

TypeError: 'NoneType' object is not subscriptable

In [615]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=df_backtestentrenamiento["timestamp"], y=df_backtestentrenamiento["capital"]))
fig.update_layout(title='Evolucion de Capital entrenamiento',
                   xaxis_title='Fecha',
                   yaxis_title='Capital')

In [616]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=df_backtestentrenamiento["timestamp"], y=df_backtestentrenamiento["balance final"]))
fig.update_layout(title='Balance General entrenamiento',
                   xaxis_title='Fecha',
                   yaxis_title='Capital')